<a href="https://colab.research.google.com/github/taasmaaawad-ops/FIRST-PROJECT/blob/main/Adversarial%20Scenario/FGSM-adversarial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [3]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# to link folder in github
!git clone https://github.com/taasmaaawad-ops/FIRST-PROJECT.git
%cd FIRST-PROJECT

Cloning into 'FIRST-PROJECT'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 33), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (75/75), 13.09 MiB | 5.17 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/FIRST-PROJECT


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from collections import defaultdict

class MultiArmedBanditThompsonSampling:
    def __init__(self, num_classifiers):
        self.num_classifiers = num_classifiers
        self.successes = defaultdict(int)
        self.failures = defaultdict(int)
        self.selected_classifier = None

    def select_classifier(self):
        max_ucb = -float('inf')
        for clf in range(self.num_classifiers):
            beta_sample = np.random.beta(self.successes[clf] + 1, self.failures[clf] + 1)
            if beta_sample > max_ucb:
                max_ucb = beta_sample
                self.selected_classifier = clf
        return self.selected_classifier

    def update(self, clf_index, success):
        if success:
            self.successes[clf_index] += 1
        else:
            self.failures[clf_index] += 1

# Load the extracted features and labels from the CSV file
extracted_features_path = r"/content/FIRST-PROJECT/encoded_features_2017.csv"
df_extracted_features = pd.read_csv(extracted_features_path)

# Separate features and labels
X = df_extracted_features.drop(columns=['Label'])
y = df_extracted_features['Label']

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize classifiers
classifiers = [
    RandomForestClassifier(n_estimators=100, random_state=42),
    LogisticRegression(max_iter=1000, random_state=42),
    SVC(kernel='linear', random_state=42),
    tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
    ])
]

# Initialize Thompson Sampling Multi-Armed Bandit
bandit = MultiArmedBanditThompsonSampling(num_classifiers=len(classifiers))

# Perform Thompson Sampling for a fixed number of rounds
num_rounds = 10
for round in range(num_rounds):
    selected_clf_index = bandit.select_classifier()
    selected_clf = classifiers[selected_clf_index]

    if isinstance(selected_clf, tf.keras.Sequential):
        # Reshape and normalize data for DNN
        X_train_dnn = np.array(X_train).reshape(-1, X_train.shape[1]).astype(np.float32) / 255.0
        X_test_dnn = np.array(X_test).reshape(-1, X_test.shape[1]).astype(np.float32) / 255.0
        # Compile the model
        selected_clf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        # Train the model
        selected_clf.fit(X_train_dnn, y_train, epochs=10, batch_size=32, validation_data=(X_test_dnn, y_test), verbose=0)
        # Predict on the testing set
        y_pred_probs = selected_clf.predict(X_test_dnn)
        y_pred = np.argmax(y_pred_probs, axis=1)
    else:
        # Train the classifier
        selected_clf.fit(X_train, y_train)
        # Predict on the testing set
        y_pred = selected_clf.predict(X_test)

    # Evaluate the selected classifier and update the bandit
    report = classification_report(y_test, y_pred, target_names=label_encoder.classes_, output_dict=True)
    accuracy = report['accuracy']
    bandit.update(selected_clf_index, accuracy)

# Generate a classification report for the best performing classifier
best_clf_index = max(bandit.successes, key=bandit.successes.get)
best_clf = classifiers[best_clf_index]

if isinstance(best_clf, tf.keras.Sequential):
    # Reshape and normalize data for DNN
    X_train_dnn = np.array(X_train).reshape(-1, X_train.shape[1]).astype(np.float32) / 255.0
    X_test_dnn = np.array(X_test).reshape(-1, X_test.shape[1]).astype(np.float32) / 255.0
    # Compile the model
    best_clf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    # Train the model
    best_clf.fit(X_train_dnn, y_train, epochs=10, batch_size=32, validation_data=(X_test_dnn, y_test), verbose=0)
    # Predict on the testing set
    y_pred_probs = best_clf.predict(X_test_dnn)
    y_pred = np.argmax(y_pred_probs, axis=1)
else:
    # Train the classifier
    best_clf.fit(X_train, y_train)
    # Predict on the testing set
    y_pred = best_clf.predict(X_test)
    # Convert label_encoder.classes_ to a list of strings
    target_names = [str(class_name) for class_name in label_encoder.classes_]

# Generate classification report for the best performing classifier
report = classification_report(y_test, y_pred, target_names=target_names)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_accuracy)
print("Classification Report:")
print(report)

FileNotFoundError: [Errno 2] No such file or directory: '/content/FIRST-PROJECT/encoded_features_2017.csv'

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support, roc_auc_score
from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import FastGradientMethod
import numpy as np
import pandas as pd

# Assuming you have trained the logistic regression model and have X_test and y_test ready
# Assuming X_test and y_test are already defined

# Step 1: Train a logistic regression model
best_clf = LogisticRegression()
best_clf.fit(X_train, y_train)

# Step 2: Convert X_test to NumPy array
X_test_np = X_test.to_numpy()

# Step 3: Generate adversarial examples using FGSM
best_clf_art = SklearnClassifier(model=best_clf, clip_values=(0, 1))
attack = FastGradientMethod(estimator=best_clf_art, eps=0.2)
X_test_adv = attack.generate(X_test_np)

# Step 4: Save the adversarial examples in a CSV file
adversarial_df = pd.DataFrame(X_test_adv, columns=[f'feature_{i}' for i in range(X_test_adv.shape[1])])
adversarial_df.to_csv('adversarial_examples_fgsml.csv', index=False)

# Step 5: Evaluate the adversarial examples
y_pred_adv = best_clf.predict(X_test_adv)
precision_adv, recall_adv, f1_score_adv, _ = precision_recall_fscore_support(y_test, y_pred_adv, average='binary')
auc_score_adv = roc_auc_score(y_test, y_pred_adv)
test_accuracy_adv = accuracy_score(y_test, y_pred_adv)
detection_rate_adv = recall_adv

print("Adversarial Test Accuracy:", test_accuracy_adv)
print("Precision on adversarial examples:", precision_adv)
print("Recall on adversarial examples:", recall_adv)
print("F1 Score on adversarial examples:", f1_score_adv)
print("AUC Score on adversarial examples:", auc_score_adv)
print("Detection Rate on adversarial examples:", detection_rate_adv)

# Generate classification report for adversarial examples
report_adv = classification_report(y_test, y_pred_adv, target_names=target_names)
print("Adversarial Classification Report:")
print(report_adv)

Adversarial Test Accuracy: 0.6621291164003913
Precision on adversarial examples: 0.7147207055858883
Recall on adversarial examples: 0.8265938069216757
F1 Score on adversarial examples: 0.7665972183118419
AUC Score on adversarial examples: 0.5764508831285791
Detection Rate on adversarial examples: 0.8265938069216757
Adversarial Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.33      0.39      4033
           1       0.71      0.83      0.77      8235

    accuracy                           0.66     12268
   macro avg       0.60      0.58      0.58     12268
weighted avg       0.64      0.66      0.64     12268



Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support, roc_auc_score
import numpy as np
import pandas as pd

# Assuming you have trained the Random Forest model and have X_test and y_test ready
# Assuming X_test and y_test are already defined

# Step 1: Train a Random Forest model
best_clf = RandomForestClassifier(n_estimators=100, random_state=42)
best_clf.fit(X_train, y_train)

# Step 2: Convert X_test to NumPy array
X_test_np = X_test.to_numpy()

# Step 3: Generate adversarial examples using FGSM
best_clf_art = SklearnClassifier(model=best_clf, clip_values=(0, 1))
attack = FastGradientMethod(estimator=best_clf_art, eps=0.2)
X_test_adv = attack.generate(X_test_np)

# Step 4: Save the adversarial examples in a CSV file
adversarial_df = pd.DataFrame(X_test_adv, columns=[f'feature_{i}' for i in range(X_test_adv.shape[1])])
adversarial_df.to_csv('adversarial_examples_fgsmrfl.csv', index=False)

# Step 5: Evaluate the adversarial examples
y_pred_adv = best_clf.predict(X_test_adv)
precision_adv, recall_adv, f1_score_adv, _ = precision_recall_fscore_support(y_test, y_pred_adv, average='binary')
auc_score_adv = roc_auc_score(y_test, y_pred_adv)
test_accuracy_adv = accuracy_score(y_test, y_pred_adv)
detection_rate_adv = recall_adv

print("Adversarial Test Accuracy:", test_accuracy_adv)
print("Precision on adversarial examples:", precision_adv)
print("Recall on adversarial examples:", recall_adv)
print("F1 Score on adversarial examples:", f1_score_adv)
print("AUC Score on adversarial examples:", auc_score_adv)
print("Detection Rate on adversarial examples:", detection_rate_adv)

# Generate classification report for adversarial examples
report_adv = classification_report(y_test, y_pred_adv, target_names=target_names)
print("Adversarial Classification Report:")
print(report_adv)

Adversarial Test Accuracy: 0.9569377990430622
Precision on adversarial examples: 0.8195488721804511
Recall on adversarial examples: 0.18227424749163879
F1 Score on adversarial examples: 0.29822161422708615
AUC Score on adversarial examples: 0.5900765846384398
Detection Rate on adversarial examples: 0.18227424749163879
Adversarial Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     11315
           1       0.82      0.18      0.30       598

    accuracy                           0.96     11913
   macro avg       0.89      0.59      0.64     11913
weighted avg       0.95      0.96      0.94     11913



SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import FastGradientMethod
import numpy as np
import pandas as pd

# Assuming you have trained the SVM model and have X_test_np and y_test ready

# Step 1: Define your SVM model
svm_model = SVC(kernel='rbf', gamma='scale', probability=True)  # Example SVM with radial basis function kernel

# Step 2: Train your SVM model
svm_model.fit(X_train, y_train)

# Step 3: Initialize a SklearnClassifier
art_classifier_svm = SklearnClassifier(model=svm_model, clip_values=(0, 1))

# Step 4: Generate adversarial examples using FGSM
attack_svm = FastGradientMethod(estimator=art_classifier_svm, eps=0.2)
X_test_adv_svm = attack_svm.generate(X_test_np)

# Step 5: Save the adversarial examples in a CSV file
adversarial_df_svm = pd.DataFrame(X_test_adv_svm, columns=[f'feature_{i}' for i in range(X_test_adv_svm.shape[1])])
adversarial_df_svm.to_csv('adversarial_examples_fgsm_svm.csv', index=False)

# Step 6: Evaluate the adversarial examples
y_pred_adv_svm = svm_model.predict(X_test_adv_svm)
report_adv_svm = classification_report(y_test, y_pred_adv_svm)
test_accuracy_adv_svm = accuracy_score(y_test, y_pred_adv_svm)
print("Adversarial Test Accuracy (SVM):", test_accuracy_adv_svm)
print("Adversarial Classification Report (SVM):")
print(report_adv_svm)

384/384 [==============================] - 1s 2ms/step
Adversarial Test Accuracy (SVM): 0.7646723182262798
Adversarial Classification Report (SVM):
              precision    recall  f1-score   support

           0       0.65      0.60      0.63      4033
           1       0.81      0.84      0.83      8235

    accuracy                           0.76     12268
   macro avg       0.73      0.72      0.73     12268
weighted avg       0.76      0.76      0.76     12268



In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import FastGradientMethod
import numpy as np
import pandas as pd

# Assuming you have trained the SVM model and have X_train and y_train ready

# Step 1: Define your SVM model
svm_model = SVC(kernel='rbf', gamma='scale', probability=True)  # Example SVM with radial basis function kernel

# Step 2: Train your SVM model
svm_model.fit(X_train, y_train)

# Step 3: Initialize a SklearnClassifier
art_classifier = SklearnClassifier(model=svm_model, clip_values=(0, 1))

# Step 4: Convert X_test to a NumPy array
X_test_np = X_test.to_numpy()

# Step 5: Generate adversarial examples using FGSM
attack = FastGradientMethod(estimator=art_classifier, eps=0.2)
X_test_adv = attack.generate(X_test_np)

# Step 6: Save the adversarial examples in a CSV file
adversarial_df = pd.DataFrame(X_test_adv, columns=[f'feature_{i}' for i in range(X_test_adv.shape[1])])
adversarial_df.to_csv('adversarial_examples_svmfgsm.csv', index=False)

# Step 7: Evaluate the adversarial examples
y_pred_adv = svm_model.predict(X_test_adv)
report_adv = classification_report(y_test, y_pred_adv)
test_accuracy_adv = accuracy_score(y_test, y_pred_adv)
precision_adv = precision_score(y_test, y_pred_adv, average='weighted')
recall_adv = recall_score(y_test, y_pred_adv, average='weighted')
f1_adv = f1_score(y_test, y_pred_adv, average='weighted')

# Calculate detection rate
detection_rate_adv = (1 - test_accuracy_adv) * 100

print("Adversarial Precision (SVM):", precision_adv)
print("Adversarial Recall (SVM):", recall_adv)
print("Adversarial F1 Score (SVM):", f1_adv)
print("Adversarial Detection Rate (SVM):", detection_rate_adv)

384/384 [==============================] - 1s 1ms/step
Adversarial Precision (SVM): 0.757271275683008
Adversarial Recall (SVM): 0.7586403651776981
Adversarial F1 Score (SVM): 0.7579103195725851
Adversarial Detection Rate (SVM): 24.135963482230196


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import FastGradientMethod
import numpy as np
import pandas as pd

# Assuming you have trained the SVM model and have X_train and y_train ready

# Step 1: Define your SVM model
svm_model = SVC(kernel='rbf', gamma='scale', probability=True)  # Example SVM with radial basis function kernel

# Step 2: Train your SVM model
svm_model.fit(X_train, y_train)

# Step 3: Initialize a SklearnClassifier
art_classifier = SklearnClassifier(model=svm_model, clip_values=(0, 1))

# Step 4: Convert X_test to a NumPy array
X_test_np = X_test.to_numpy()

# Step 5: Generate adversarial examples using FGSM
attack = FastGradientMethod(estimator=art_classifier, eps=0.2)
X_test_adv = attack.generate(X_test_np)

# Step 6: Save the adversarial examples in a CSV file
adversarial_df = pd.DataFrame(X_test_adv, columns=[f'feature_{i}' for i in range(X_test_adv.shape[1])])
adversarial_df.to_csv('adversarial_examples_svmfgsm.csv', index=False)

# Step 7: Evaluate the adversarial examples
y_pred_adv = svm_model.decision_function(X_test_adv)  # Using decision function for SVM
auc_score_adv = roc_auc_score(y_test, y_pred_adv)

print("Adversarial AUC Score (SVM):", auc_score_adv)

384/384 [==============================] - 1s 3ms/step
Adversarial AUC Score (SVM): 0.7483838929921048


DNN

In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod
import numpy as np
import pandas as pd

# Assuming you have trained the DNN model and have X_test_np and y_test ready
num_classes = 10  # Assuming num_classes is defined
input_shape = X_train.shape[1]  # Assuming X_train is defined

# Step 1: Define your DNN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(input_shape,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 2: Train your DNN model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Step 3: Generate adversarial examples using FGSM
art_classifier = TensorFlowV2Classifier(model=model, nb_classes=num_classes, input_shape=input_shape, clip_values=(0, 1), loss_object=tf.keras.losses.SparseCategoricalCrossentropy())
attack = FastGradientMethod(estimator=art_classifier, eps=0.2)
X_test_adv = attack.generate(X_test_np)

# Step 4: Save the adversarial examples in a CSV file
adversarial_df = pd.DataFrame(X_test_adv, columns=[f'feature_{i}' for i in range(X_test_adv.shape[1])])
adversarial_df.to_csv('adversarial_examples_fgsmdnn.csv', index=False)

# Step 5: Evaluate the adversarial examples
y_pred_adv = np.argmax(model.predict(X_test_adv), axis=1)
report_adv = classification_report(y_test, y_pred_adv)
test_accuracy_adv = accuracy_score(y_test, y_pred_adv)
print("Adversarial Test Accuracy (DNN):", test_accuracy_adv)
print("Adversarial Classification Report (DNN):")
print(report_adv)

Epoch 1/10
1227/1227 [==============================] - 4s 2ms/step - loss: 0.2656 - accuracy: 0.9162 - val_loss: 0.1167 - val_accuracy: 0.9551
Epoch 2/10
1227/1227 [==============================] - 3s 2ms/step - loss: 0.1155 - accuracy: 0.9640 - val_loss: 0.1529 - val_accuracy: 0.9679
Epoch 3/10
1227/1227 [==============================] - 3s 2ms/step - loss: 0.1097 - accuracy: 0.9711 - val_loss: 0.0753 - val_accuracy: 0.9787
Epoch 4/10
1227/1227 [==============================] - 4s 3ms/step - loss: 0.0851 - accuracy: 0.9740 - val_loss: 0.0956 - val_accuracy: 0.9759
Epoch 5/10
1227/1227 [==============================] - 3s 2ms/step - loss: 0.0786 - accuracy: 0.9765 - val_loss: 0.0803 - val_accuracy: 0.9798
Epoch 6/10
1227/1227 [==============================] - 3s 2ms/step - loss: 0.0709 - accuracy: 0.9778 - val_loss: 0.0630 - val_accuracy: 0.9797
Epoch 7/10
1227/1227 [==============================] - 3s 2ms/step - loss: 0.0644 - accuracy: 0.9796 - val_loss: 0.0611 - val_accuracy:

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision, recall, and F1 score for adversarial examples
precision_adv = precision_score(y_test, y_pred_adv, average='weighted')
recall_adv = recall_score(y_test, y_pred_adv, average='weighted')
f1_adv = f1_score(y_test, y_pred_adv, average='weighted')

# Calculate precision, recall, and F1 score for non-adversarial examples
precision_non_adv = precision_score(y_test, y_pred_non_adv, average='weighted')
recall_non_adv = recall_score(y_test, y_pred_non_adv, average='weighted')
f1_non_adv = f1_score(y_test, y_pred_non_adv, average='weighted')

# Calculate detection rate for both adversarial and non-adversarial examples
detection_rate_adv = 1 - test_accuracy_adv

print("Adversarial Precision (DNN):", precision_adv)
print("Adversarial Recall (DNN):", recall_adv)
print("Adversarial F1 Score (DNN):", f1_adv)
print("Detection Rate for Adversarial Examples (DNN):", detection_rate_adv)

Adversarial Precision (DNN): 0.535011878624934
Adversarial Recall (DNN): 0.46380828170850996
Adversarial F1 Score (DNN): 0.4757323656653002
Detection Rate for Adversarial Examples (DNN): 0.53619171829149


In [ ]:
from sklearn.metrics import roc_auc_score

# Calculate AUC score for adversarial examples
auc_score_adv = roc_auc_score(y_test, y_pred_adv)
print("Adversarial AUC Score (DNN):", auc_score_adv)

Adversarial AUC Score (DNN): 0.5928594717965437


IDS-Anta

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import os

class MultiArmedBanditAntColonyOptimization:
    def __init__(self, n_arms, n_ants):
        self.n_arms = n_arms
        self.n_ants = n_ants
        self.arms = [SVC(probability=True), LogisticRegression(), RandomForestClassifier(), self.create_dnn_model()]
        self.pheromone = np.ones(n_arms)
        self.epsilon = 0.1

    def create_dnn_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def choose_arm(self):
        probabilities = self.pheromone / np.sum(self.pheromone)
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.n_arms)
        else:
            return np.random.choice(self.n_arms, p=probabilities)

    def update(self, arm, reward):
        self.pheromone[arm] += reward

# Fast Gradient Sign Method (FGSM) for generating adversarial samples
def generate_adversarial_sample(model, X, y, epsilon=0.1):
    with tf.GradientTape() as tape:
        tape.watch(X)
        prediction = model(X)
        loss = tf.keras.losses.binary_crossentropy(y, prediction)
    gradient = tape.gradient(loss, X)
    signed_grad = tf.sign(gradient)
    adversarial_X = X + epsilon * signed_grad
    return adversarial_X

# Load the extracted features and labels from the CSV file
extracted_features_path = r"E:\train\encoded_features_2017_18.csv"
df_extracted_features = pd.read_csv(extracted_features_path)

# Separate features and labels
X = df_extracted_features.drop(columns=['Label'])
y = df_extracted_features['Label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Taining we use the 2017&18 Data and testing we take the 2017 adversial data
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize Multi-Armed Bandit with Ant Colony Optimization
n_arms = 4  # Number of classifiers
n_ants = 10  # Number of ants
bandit = MultiArmedBanditAntColonyOptimization(n_arms, n_ants)

# Train the Multi-Armed Bandit
num_iterations = 10
for _ in range(num_iterations):
    for _ in range(n_ants):
        arm = bandit.choose_arm()
        classifier = bandit.arms[arm]
        if classifier.__class__.__name__ != 'Sequential':
            # For non-DNN classifiers
            classifier.fit(X_train, y_train)
            y_pred = classifier.predict(X_train)
            reward = accuracy_score(y_train, y_pred)
            bandit.update(arm, reward)
        else:
            # For DNN classifier using FGSM for adversarial training
            adversarial_X_train = generate_adversarial_sample(classifier, X_train, y_train)
            classifier.fit(adversarial_X_train, y_train)
            _, accuracy = classifier.evaluate(X_train, y_train, verbose=0)
            reward = accuracy
            bandit.update(arm, reward)

# Choose the best classifier
best_arm = np.argmax(bandit.pheromone)
best_classifier = bandit.arms[best_arm]

# Evaluate the best classifier on the test set
if best_classifier.__class__.__name__ != 'Sequential':
    y_pred = best_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)
else:
    _, accuracy = best_classifier.evaluate(X_test, y_test, verbose=0)
    y_pred = (best_classifier.predict(X_test) > 0.5).astype("int32")
    classification_rep = classification_report(y_test, y_pred)

# Print accuracy and classification report for the original test set
print("Results for original test set:")
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

# Generate and save adversarial examples for the test set using FGSM
adversarial_X_test = generate_adversarial_sample(best_classifier, X_test, y_test)
adversarial_samples_dir = "adversarial_samples"
os.makedirs(adversarial_samples_dir, exist_ok=True)
np.savetxt(os.path.join(adversarial_samples_dir, "adversarial_samples.csv"), adversarial_X_test, delimiter=",")

# Evaluate the best classifier on the adversarial test set
if best_classifier.__class__.__name__ != 'Sequential':
    y_pred_adv = best_classifier.predict(adversarial_X_test)
    accuracy_adv = accuracy_score(y_test, y_pred_adv)
    classification_rep_adv = classification_report(y_test, y_pred_adv)
else:
    _, accuracy_adv = best_classifier.evaluate(adversarial_X_test, y_test, verbose=0)
    y_pred_adv = (best_classifier.predict(adversarial_X_test) > 0.5).astype("int32")
    classification_rep_adv = classification_report(y_test, y_pred_adv)

# Print accuracy and classification report for the adversarial test set
print("\nResults for adversarial test set:")
print(f"Accuracy: {accuracy_adv}")
print("Classification Report:")
print(classification_rep_adv)

356/356 [==============================] - 1s 1ms/step
Accuracy: 0.9996485114097595
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       282
           1       1.00      1.00      1.00     11099

    accuracy                           1.00     11381
   macro avg       0.99      1.00      1.00     11381
weighted avg       1.00      1.00      1.00     11381



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import SklearnClassifier, TensorFlowClassifier

class MultiArmedBanditAntColonyOptimization:
    def __init__(self, n_arms, n_ants):
        self.n_arms = n_arms
        self.n_ants = n_ants
        self.arms = [SVC(probability=True), LogisticRegression(), RandomForestClassifier(), self.create_dnn_model()]
        self.pheromone = np.ones(n_arms)
        self.epsilon = 0.1

    def create_dnn_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def choose_arm(self):
        probabilities = self.pheromone / np.sum(self.pheromone)
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.n_arms)
        else:
            return np.random.choice(self.n_arms, p=probabilities)

    def update(self, arm, reward):
        self.pheromone[arm] += reward

# Load the extracted features and labels from the CSV file
extracted_features_path = r"E:\train\encoded_features_2017_18.csv"
df_extracted_features = pd.read_csv(extracted_features_path)

# Separate features and labels
X = df_extracted_features.drop(columns=['Label'])
y = df_extracted_features['Label']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Taining we use the 2017&18 Data and testing we take the 2017 adversial data
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize Multi-Armed Bandit with Ant Colony Optimization
n_arms = 4  # Number of classifiers
n_ants = 10  # Number of ants
bandit = MultiArmedBanditAntColonyOptimization(n_arms, n_ants)

# Train the Multi-Armed Bandit
num_iterations = 10
for _ in range(num_iterations):
    for _ in range(n_ants):
        arm = bandit.choose_arm()
        classifier = bandit.arms[arm]
        classifier.fit(X_train, y_train)
        if classifier.__class__.__name__ != 'Sequential':
            y_pred = classifier.predict(X_train)
            reward = accuracy_score(y_train, y_pred)
        else:
            _, accuracy = classifier.evaluate(X_train, y_train, verbose=0)
            reward = accuracy
        bandit.update(arm, reward)

# Choose the best classifier
best_arm = np.argmax(bandit.pheromone)
best_classifier = bandit.arms[best_arm]

# Create ART classifier for the best classifier
if best_classifier.__class__.__name__ != 'Sequential':
    art_classifier = SklearnClassifier(model=best_classifier)
else:
    art_classifier = TensorFlowClassifier(model=self.create_dnn_model(), nb_classes=2, input_shape=X_train.shape[1])

# Generate adversarial samples using Fast Gradient Method (FGM)
fgm_attack = FastGradientMethod(estimator=art_classifier, eps=0.1)
X_test_adv = fgm_attack.generate(X_test)

# Evaluate the best classifier on the adversarial test set
y_pred_adv = best_classifier.predict(X_test_adv)
precision_adv, recall_adv, f1_score_adv, _ = precision_recall_fscore_support(y_test, y_pred_adv, average='binary')
auc_score_adv = roc_auc_score(y_test, y_pred_adv)

# Compute detection rate on adversarial samples (equivalent to recall)
detection_rate_adv = recall_adv

# Print precision, recall, detection rate, F1 score, and AUC score on adversarial samples
print("Precision on adversarial samples:", precision_adv)
print("Recall on adversarial samples:", recall_adv)
print("Detection Rate on adversarial samples:", detection_rate_adv)
print("F1 Score on adversarial samples:", f1_score_adv)
print("AUC Score on adversarial samples:", auc_score_adv)

Precision on adversarial samples: 0.9883333333333333
Recall on adversarial samples: 0.9857265484877259
Detection Rate on adversarial samples: 0.9418545945625773
F1 Score on adversarial samples: 0.9899497487437185
AUC Score on adversarial samples: 0.9894085454978075
